In [ ]:
month_34 = all_data[all_data.date_block_num == 34].fillna(0)
month_34['item_cnt_month'] = full_model2.predict(month_34[predictors])
test = pd.merge(test_df, month_34, on=["shop_id","item_id"]).fillna(0)

test = test.loc[:,['ID', 'item_cnt_month']]
test.set_index("ID", inplace=True)
test["item_cnt_month"] = test["item_cnt_month"].round().astype("int64")

test[test.item_cnt_month>20] = 20
test.to_csv("shop_item_trends.csv")

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor
full_model2 = RandomForestRegressor(n_estimators=64, max_features=4, n_jobs=6) #len(predictors)/2
full_model2.fit(train[predictors], train.target)

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
model2 = RandomForestRegressor(n_estimators=64, max_features=4, n_jobs=4)
model2.fit(X_train, y_train)
ypred = model2.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, ypred))
print("RMSE: %f" % (rmse))

In [ ]:
train = all_data[(all_data.date_block_num>12)&(all_data.date_block_num<34)].fillna(0)

predictors += ['item_trend', 'item_seasonal', 'item_forecast'] 

X_train = train[train.date_block_num <  33][predictors]
X_valid = train[train.date_block_num == 33][predictors]
y_train = train[train.date_block_num <  33].target
y_valid = train[train.date_block_num == 33].target

In [ ]:
all_data = pd.merge(all_data, extrapolate_item, how='left', on=['date_block_num','item_id'])

In [ ]:
import gc
del month_34
gc.collect()

In [ ]:
item_monthly_sales[item_monthly_sales.item_id==little_data_items[0]]

In [ ]:
# force all out-of-stock items to yeild zero
for item in no_stock_items:

        extrapolate_item = pd.concat([extrapolate_item,
                             pd.DataFrame({
                                 'date_block_num':  range(35),
                                 'item_id':         [item]*35,
                                 'item_trend':      [0]*35,
                                 'item_seasonal':   [0]*35,
                                 'item_forecast':   [0]*35
                             })])

for item in little_data_items:

    sales = item_monthly_sales[item_monthly_sales.item_id==item]

    target_33 = 33 in sales.index and sales.target[33] or 0
    target_32 = 32 in sales.index and sales.target[32] or 0
    target_31 = 31 in sales.index and sales.target[31] or 0

    forecast = (target_33+target_32+target_31)/3.

    extrapolate_item = pd.concat([extrapolate_item,
                             pd.DataFrame({
                                 'date_block_num':  [34],
                                 'item_id':         [item],
                                 'item_trend':      0,
                                 'item_seasonal':   0,
                                 'item_forecast':   forecast
                             })])


In [ ]:
prev_year    = extrapolate_item[extrapolate_item.date_block_num==(34-12)][['item_id','item_seasonal']]
prev_month_0 = extrapolate_item[extrapolate_item.date_block_num==33][['item_id','item_trend']]
prev_month_1 = extrapolate_item[extrapolate_item.date_block_num==32][['item_id','item_trend']]
prev_month_2 = extrapolate_item[extrapolate_item.date_block_num==31][['item_id','item_trend']]

prev_month = pd.DataFrame()

for item in prev_month_0.item_id.unique()[1:10]:
    t0 = prev_month_0[prev_month_0.item_id == item].item_trend.values[0]
    t1 = prev_month_1[prev_month_1.item_id == item].item_trend.values[0]
    t2 = prev_month_2[prev_month_2.item_id == item].item_trend.values[0]

    fit = np.polyfit([31,32,33], [t2,t1,t0] ,1) # 1 is for linear fit
    line = np.poly1d(fit)

    prev_month = pd.concat([prev_month, pd.DataFrame({'item_id': [item], 'item_trend': [line(34)]})])

    if item%500 == 0:
        plt.plot([t2,t1,t0,line(34)], label='Trend')


month_34 = pd.merge(prev_month, prev_year, how='inner', on=['item_id'])
month_34['date_block_num'] = 34
month_34['item_forecast'] = month_34['item_trend'] + month_34['item_seasonal']

extrapolate_item = pd.concat([extrapolate_item,month_34])
extrapolate_item.shape

In [ ]:
plt.hist([len(no_stock_items),len(little_data_items)])

In [ ]:
%%time
#import matplotlib.pyplot as plt
#%matplotlib inline

item_monthly_sales = monthly.groupby(["item_id","date_block_num"]).target.sum().reset_index("item_id")

extrapolate_item = pd.DataFrame()

no_stock_items = []

little_data_items = []

for item in item_monthly_sales.item_id.unique():
    
    sales = item_monthly_sales[item_monthly_sales.item_id==item]
    sales_ts = pd.merge(timeline[0:-1], sales, how='left', on=['date_block_num'])

    # some items are discuntinued or go out of stock, don't forecast those yet
    if sum( np.isnan(sales_ts[-3:].target) ) == 3: # >0 ?
        no_stock_items.append(item)
        continue

    # at this point last observations are not zeros, let's bridge the gaps
    sales_ts.target = sales_ts.target.interpolate()
    # last missing values (if any) are automatically imputed with their lags above

    # any remaining nulls are from earliy continuous periods (interpolate wont impute those)
    if any(np.isnan(sales_ts.target)):
        last_null = max( np.where(np.isnan(sales_ts.target))[0] )
        sales_ts = sales_ts[last_null+1:]

    if len(sales_ts) <= 12:
        little_data_items.append(item)
        continue

    ts = pd.Series(sales_ts.target.values, index=sales_ts["begin_date"])

    if any( ts.isnull() ):
        print("Problem")
        break

    decomposition = seasonal_decompose(ts, extrapolate_trend=1, freq=12)

    trend    = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    
    if item%500 == 0:
        plt.plot(trend, label='Trend')

    extrapolate_item = pd.concat([extrapolate_item,
                             pd.DataFrame({
                                 'date_block_num':  sales_ts.date_block_num,
                                 'item_id':         [item]*len(ts),
                                 'item_trend':      trend.values,
                                 'item_seasonal':   seasonal.values,
                                 'item_forecast':   trend.values + seasonal.values
                             })])
